In [1]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

Save path for evaluation reports

In [2]:
results_path = "./results"

# Load training data

Load pickled embedded training data

In [44]:
path_prefix = "./../../get_text_detect_space/datasets/embed/"
dataset_path = "webtext.train.jsonl.clean100k.csv+xl-1542M.train.jsonl.clean100k.csv_embed.pickle"

In [45]:
with open(os.path.join(path_prefix, dataset_path), "rb") as f:
    training_data = pickle.load(f)

X = training_data["X"]
y = training_data["y"]

In [46]:
X

array([[ 1.56034485e-01, -1.03202422e-03,  9.29313446e-02, ...,
        -6.20653304e-03,  1.78792963e-02, -1.16704679e-02],
       [ 1.19502099e-01,  1.64194560e-02, -1.96907246e-02, ...,
        -1.13290602e-02,  5.91688235e-03,  1.52714059e-03],
       [ 1.67067327e-01,  5.72166997e-02, -9.31411220e-03, ...,
        -6.56531715e-03, -1.25897509e-02,  2.12441978e-03],
       ...,
       [ 1.45809149e-01, -3.29254096e-02, -2.40912571e-02, ...,
         6.88818067e-03, -4.50552379e-03, -3.19229779e-03],
       [ 1.07385538e-01, -5.34839545e-02,  2.93454104e-02, ...,
        -1.70121954e-02, -6.23178832e-03,  8.76551081e-03],
       [ 1.83371955e-01,  9.32380781e-02, -1.46499623e-03, ...,
         1.69348226e-05, -6.23653770e-03,  8.20869394e-03]])

In [47]:
base_prefix_path = "./../../get_text_detect_space/datasets"
tfidf_path = f"{base_prefix_path}/embed/webtext.train.jsonl.clean100k.csv+xl-1542M.train.jsonl.clean100k.csv_embed.pickle"
tag_path = f"{base_prefix_path}/tag/embed/webtext_xl-1542M.tag_embed.pickle"

with open(tfidf_path, "rb") as f:
    training_data = pickle.load(f)

X = training_data["X"]
y = training_data["y"]

with open(tag_path, "rb") as f:
    training_data = pickle.load(f)
    
X = np.concatenate([X, training_data["X"]], axis=1)


In [48]:
X.shape

(200000, 600)

# Load test data

Load pickled embedded test data

In [49]:
path_prefix = "./../../get_text_detect_space/datasets/embed/"
dataset_names = ["webtext.test.human_embed.pickle", "gpt2.xl-1542M.test.machine_embed.pickle"]

In [50]:
with open(os.path.join(path_prefix, dataset_names[0]), "rb") as f:
    test_data = pickle.load(f)
    X_test = test_data["X"]
    y_test = test_data["y"]
if len(dataset_names)>1:
    for dataset_name in dataset_names[1:]:
        with open(os.path.join(path_prefix, dataset_name), "rb") as f:
            test_data = pickle.load(f)
            X_test = np.concatenate((X_test, test_data["X"]))
            y_test += test_data["y"]

In [51]:
path_prefix = "./../../get_text_detect_space/datasets/tag/embed/"
dataset_name = "webtext.test_xl-1542M.test.tag_embed.pickle"
with open(os.path.join(path_prefix, dataset_name), "rb") as f:
    test_data = pickle.load(f)
    X_test = np.concatenate([X_test, test_data["X"]], axis=1)

In [52]:
X_test

array([[ 0.036449  ,  0.01040318, -0.01484086, ..., -0.00727127,
         0.00040762,  0.00319771],
       [ 0.1697334 ,  0.00916163,  0.08250503, ..., -0.00558962,
        -0.01145208,  0.00735047],
       [ 0.08371905, -0.0375891 ,  0.01756523, ...,  0.00710054,
        -0.01029539, -0.02777718],
       ...,
       [ 0.16086878, -0.10212953, -0.00653402, ..., -0.00563827,
         0.01246122, -0.01094326],
       [ 0.14679124, -0.05596965,  0.00043219, ...,  0.01002533,
         0.01669157,  0.00908132],
       [ 0.13941275, -0.04264188,  0.0272543 , ..., -0.02123425,
         0.00149337, -0.01113054]])

In [53]:
X_test.shape

(10000, 600)

# Train classifiers

In [54]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

## Log Reg

In [55]:
logreg = LogisticRegression(solver='liblinear')
# parameters = {'C': [1/8, 1, 8]}
# parameters = {'C': [1/64, 1/32, 1/16, 1/8, 1/4, 1/2, 1, 2, 4, 8, 16, 32, 64]}
parameters = {'C': [1]}

clf_logreg = GridSearchCV(logreg, parameters, n_jobs=-1)

In [56]:
clf_logreg.fit(X, y)

GridSearchCV(estimator=LogisticRegression(solver='liblinear'), n_jobs=-1,
             param_grid={'C': [0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5, 1,
                               2, 4, 8, 16, 32, 64]})

Save cross-validation results

In [59]:
df_clf_logreg = pd.DataFrame(clf_logreg.cv_results_)

df_clf_logreg.to_csv(os.path.join(results_path,"logreg_cv_tfidftag_results_extendedparamspace.csv"))
df_clf_logreg

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,31.566443,1.014928,0.180223,0.003066,0.015625,{'C': 0.015625},0.679875,0.676700,0.674975,0.675800,0.677800,0.677030,0.001705,13
1,38.082305,0.976354,0.166000,0.013313,0.03125,{'C': 0.03125},0.687075,0.685925,0.682450,0.684875,0.685850,0.685235,0.001557,12
2,42.168504,1.149418,0.154231,0.012126,0.0625,{'C': 0.0625},0.693925,0.693825,0.691975,0.693650,0.694150,0.693505,0.000782,11
3,59.376529,2.619354,0.151162,0.002688,0.125,{'C': 0.125},0.700925,0.702275,0.700875,0.702450,0.701400,0.701585,0.000663,10
4,67.468253,0.887103,0.147013,0.007192,0.25,{'C': 0.25},0.708200,0.709775,0.708975,0.708825,0.708700,0.708895,0.000511,9
5,78.785817,1.959728,0.148136,0.004836,0.5,{'C': 0.5},0.715125,0.716050,0.715375,0.715775,0.716350,0.715735,0.000443,8
6,85.458050,4.354527,0.147224,0.006981,1,{'C': 1},0.720550,0.721800,0.720250,0.721650,0.720950,0.721040,0.000604,7
7,98.690702,8.611689,0.119810,0.027326,2,{'C': 2},0.725575,0.725225,0.723675,0.726100,0.723775,0.724870,0.000976,6
8,110.466402,6.979461,0.096045,0.018917,4,{'C': 4},0.728125,0.726900,0.724950,0.728900,0.726575,0.727090,0.001359,5
9,103.675841,3.815306,0.087199,0.025652,8,{'C': 8},0.729775,0.728175,0.726150,0.729200,0.727075,0.728075,0.001332,4


Evaluate best-classifier on test data

In [60]:
y_predict = clf_logreg.predict(X_test)
df_cr = pd.DataFrame(classification_report(y_test, y_predict, output_dict=True))
df_cr.to_csv(os.path.join(results_path,"logreg_test_tfidftag_results_extendedparamspace.csv"))
df_cr

,0,1,accuracy,macro avg,weighted avg
precision,0.724611,0.714425,0.7194,0.719518,0.719518
recall,0.707800,0.731000,0.7194,0.719400,0.719400
f1-score,0.716107,0.722618,0.7194,0.719362,0.719362
support,5000.000000,5000.000000,0.7194,10000.000000,10000.000000


## GradientBoosting

In [80]:
from sklearn.ensemble import GradientBoostingClassifier

In [81]:
gboost = GradientBoostingClassifier(random_state=42)
parameters = {'n_estimators':[100, 200, 300], 'max_depth':[3, 4, 5]}
# parameters = {'n_estimators':[100, 200, 300], 'max_depth':[3, 4, 5], 'learning_rate':[0.2, 0.5, 1.0]}

clf_gboost = GridSearchCV(gboost, parameters, n_jobs=-1)

In [ ]:
clf_gboost.fit(X, y)

Save cross-validation results

In [ ]:
df_clf_gboost = pd.DataFrame(clf_gboost.cv_results_)

df_clf_gboost.to_csv(os.path.join(results_path,"gboost_cv_tag_results.csv"))
df_clf_gboost

Evaluate best-classifier on test data

In [ ]:
y_predict = clf_gboost.predict(X_test)
df_cr = pd.DataFrame(classification_report(y_test, y_predict, output_dict=True))
df_cr.to_csv(os.path.join(results_path,"gboost_test_tag_results.csv"))
df_cr

## SVM

In [67]:
from sklearn.svm import SVC

In [68]:
svm = SVC(random_state=42)
#parameters = {'C':[0.5, 1.0, 2.0, 3.0], 'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'degree':[1, 2, 3],
#             'coef0':[0.0, 0.5, 1.0]}
# parameters = {'C':[0.5, 1.0, 2.0, 3.0]}
parameters = {'C':[1.0]}

clf_svm = GridSearchCV(svm, parameters, n_jobs=-1)

In [ ]:
clf_svm.fit(X, y)

Save cross-validation results

In [ ]:
df_clf_svm = pd.DataFrame(clf_svm.cv_results_)

df_clf_svm.to_csv(os.path.join(results_path,"svm_cv_results.csv"))
df_clf_svm

Evaluate best-classifier on test data

In [ ]:
y_predict = clf_svm.predict(X_test)
df_cr = pd.DataFrame(classification_report(y_test, y_predict, output_dict=True))
df_cr.to_csv(os.path.join(results_path,"svm_test_results.csv"))
df_cr

## Random forest

In [61]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
# parameters = {'n_estimators':[100, 200, 300], 'criterion':['gini', 'entropy', 'log_loss']}
parameters = {'n_estimators':[300], 'criterion':['gini']}

clf_rf = GridSearchCV(rf, parameters, n_jobs=-1)

In [63]:
clf_rf.fit(X, y)

GridSearchCV(estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini'], 'n_estimators': [300]})

Save cross-validation results

In [64]:
df_clf_rf = pd.DataFrame(clf_rf.cv_results_)

df_clf_rf.to_csv(os.path.join(results_path,"randomforest_cv_results.csv"))
df_clf_rf

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,375.550868,1.511019,0.675215,0.149193,gini,300,"{'criterion': 'gini', 'n_estimators': 300}",0.6791,0.676375,0.67695,0.6766,0.678175,0.67744,0.001037,1


Evaluate best-classifier on test data

In [65]:
y_predict = clf_rf.predict(X_test)
df_cr = pd.DataFrame(classification_report(y_test, y_predict, output_dict=True))
df_cr.to_csv(os.path.join(results_path,"randomforest_test_results.csv"))
df_cr

,0,1,accuracy,macro avg,weighted avg
precision,0.665680,0.695122,0.6792,0.680401,0.680401
recall,0.720000,0.638400,0.6792,0.679200,0.679200
f1-score,0.691776,0.665555,0.6792,0.678665,0.678665
support,5000.000000,5000.000000,0.6792,10000.000000,10000.000000
